In [ ]:
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://turkiye.tvsmotor.com/tr/dealer-locator")
driver.execute_script("document.body.style.zoom='33%'")

wait = WebDriverWait(driver, 15)

def wait_and_click(css_selector, timeout=10, max_retries=3):
    for attempt in range(max_retries):
        try:
            el = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
            )

            # Loader varsa onu bekle
            try:
                WebDriverWait(driver, 3).until(
                    EC.invisibility_of_element_located((By.CLASS_NAME, "full-page-loader"))
                )
            except:
                pass  # Loader görünmüyorsa geç

            driver.execute_script("arguments[0].scrollIntoView(true);", el)
            el.click()
            return el
        except Exception as e:
            print(f"[{attempt+1}. deneme] Hata oluştu: {e}")
            time.sleep(1)
    raise Exception(f"{max_retries} kez denendi, ancak tıklama başarısız oldu.")


def get_options():
    return wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.select2-results__option")))

try:
    results = []
    results.append(["İl", "İlçe", "Bayi Adı", "Telefon 1", "Telefon 2", "Telefon 3"])


    # 1. İL SEÇ
    wait_and_click("#select2-state-container")
    iller = get_options()
    wait_and_click("#select2-state-container")
    i=0
    for il_el in iller:
        wait_and_click("#select2-state-container")
        iller = get_options()
        
        il = iller[i].text.strip()

        # Arama kutusuna yaz
        input_box = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@class='select2-search__field']")))
        input_box.clear()
        input_box.send_keys(il)

        # İlk sonucu tıkla
        il_secim = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.select2-results__option")))
        il_secim.click()

        i=i+1
        # 2. İLÇE SEÇ
        wait_and_click("#select2-city-container")
        ilceler = get_options()
        wait_and_click("#select2-city-container")
        j=0
        for ilce_el in ilceler:
            
            wait_and_click("#select2-city-container")
            ilceler = get_options()

            ilce = ilceler[j].text.strip()
            if(il == "IĞDIR" or il=="KİLİS" or il=="OTHER"):
                continue

            # Arama kutusunu bul ve yaz
            input_box = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='senna_surface1']/span/span/span[1]/input")))
            input_box.clear()
            input_box.send_keys(ilce)  # İlçe adını yaz

            # İlk çıkan ilçe seçeneğini tıkla
            ilce_secim = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='select2-city-results']/li[1]")))
            ilce_secim.click()

            j=j+1
            # 3. SERVİS SEÇ
            wait_and_click("#select2-dealer-container")
            turler = get_options()
            for tur_el in turler:
                tur_el.click()
                break

            # 4. GÖNDER BUTONU
            gonder_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#submit-btn")))
            gonder_btn.click()

            # 5. VERİ ÇEK

            # Bayi sayısını kontrol et
            bayi_sayisi_element = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='dealer-listing']/div/div[1]/span")))

            # Eğer bayi sayısı 0 ise, bu durumda atla
            bayi_sayisi_text = bayi_sayisi_element.text.strip()

            if bayi_sayisi_text[0] == "0":
                continue  # Döngüyü atlayıp sonraki ilçeye geç

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#dealer-listing .dealer-location h3")))

            bayi_adlari = driver.find_elements(By.CSS_SELECTOR, "#dealer-listing .dealer-location h3")
            telefonlar = driver.find_elements(By.CSS_SELECTOR, ".dealer-phone a")

            for bayi, tel in zip(bayi_adlari, telefonlar):
                # Telefonu temizle: boşlukları kaldır, - ve , ile ayır, liste yap
                temiz_tel= re.split(r"[,\-/]", tel.text)
                temiz_tel = [
                    re.sub(r"[()\s]", "", str(t))
                    for t in temiz_tel
                    if str(t).strip()
                ]
                temiz_tel = [int(t) if t.isdigit() else 0 for t in temiz_tel]

                # Gerekirse boşlukları eşitle (3 telefon numarası olacak şekilde)
                while len(temiz_tel) < 3:
                    temiz_tel.append(0)

                results.append([
                    il.strip(),            # İl
                    ilce.strip(),          # İlçe
                    bayi.text.strip(),     # Bayi Adı
                    temiz_tel[0],          # Tel1
                    temiz_tel[1],          # Tel2
                    temiz_tel[2]           # Tel3
                ])

    
except Exception as e:
    print(e , il,ilce,bayi.text, temiz_tel)

# 7. SONUÇLARI YAZDIR
for r in results:
    print(r)

driver.quit()


In [ ]:
import openpyxl
import copy
result1 = copy.deepcopy(results)
for i in range(1, len(result1)):
    for j in range(3, 6):  # Telefon 1, 2, 3 sütunları
        value = result1[i][j]

        if isinstance(value, str):
            result1[i][j] = int(value)
        elif value == 0:
            result1[i][j] = ""
        print(result1[i][j])



# Yeni Excel dosyası oluştur
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "TVS RAİDER BAYİ LİSTESİ"

# result1 listesindeki tüm satırları yaz
for row in result1:
    ws.append(row)

# Dosyayı kaydet
wb.save("TVS-RAİDER-BAYİ-LİSTES.xlsx")
print("Excel dosyası oluşturuldu: bayi_verileri.xlsx")
